## Making the Wikipedia Tool here


In [1]:
# !pip install arxiv


In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)



In [3]:
wiki_tool.name

'wikipedia'

In [6]:
# wiki_tool.run("Tell me about the Machine Learning")

- Researching of material

In [4]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://docs.smith.langchain.com/"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()
docs[0]

USER_AGENT environment variable not set, consider setting it to identify your requests.
Fetching pages: 100%|##########| 1/1 [00:00<00:00,  4.76it/s]


Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='<!doctype html>\n<html lang="en" dir="ltr" class="docs-wrapper plugin-docs plugin-id-default docs-version-current docs-doc-page docs-doc-id-index" data-has-hydrated="false">\n<head>\n<meta charset="UTF-8">\n<meta name="generator" content="Docusaurus v3.5.2">\n<title data-rh="true">Get started with LangSmith | 🦜️🛠️ LangSmith</title><meta data-rh="true" name="viewport" content="width=device-width,initial-scale=1"><meta data-rh="true" name="twitter:card" content="summary_large_image"><meta data-rh="true" property="og:image" content="https://docs.smith.langchain.com/img/langsmith-preview.png"><meta data-rh="true" name="twitter:image" content="https://docs.smith.langchain.com/img/langsmith-preview.png"><meta data-rh="true" property="og:url" content="

In [6]:
from langchain_community.document_transformers import Html2TextTransformer

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)
docs_transformed

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Skip to main content\n\nLearn the essentials of LangSmith in the new Introduction to LangSmith course!\nEnroll for free.\n\nAPI Reference\n\n  * REST\n  * Python\n\nSearch\n\nRegion\n\n  * US\n  * EU\n\nGo to App\n\n  * Quick Start\n  * Observability\n\n  * Evaluation\n\n  * Prompt Engineering\n\n  * Deployment (LangGraph Platform)\n  * * * *\n\n  * Administration\n\n  * Self-hosting\n\n  * Pricing\n  * * * *\n\n  * Reference\n\n    * Cloud architecture and scalability\n    * Authz and Authn\n\n      * Authentication methods\n    * data_formats\n\n    * Evaluation\n\n      * Dataset transformations\n    * Regions FAQ\n    * sdk_reference\n\n  *   * Quick Start\n\nOn this page\n\n# Get started with LangSmith\n\n**LangSmith** is a platform for bu

## Making the web base loader get the info store in vector db for embedding

In [13]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import JinaEmbeddings


In [7]:
from dotenv import  load_dotenv
import os 
load_dotenv()

os.environ['jina_api_key'] = os.getenv('jina_api_key')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')

In [8]:
loader = WebBaseLoader('https://docs.smith.langchain.com/')
docs = loader.load()

In [24]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
documents = splitter.split_documents(docs)
# documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceQuick StartOn this pageGet started with LangSmith\nLangSmith is a platform for building production-grade LLM applications.\nIt allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.\nWith LangSmith you can:'),
 Docum

In [10]:
embedding = JinaEmbeddings(
    jina_api_key=os.getenv('jina_api_key'), model_name="jina-embeddings-v3"
)

In [25]:
embedding

JinaEmbeddings(session=<requests.sessions.Session object at 0x78f1fd5e3680>, model_name='jina-embeddings-v3', jina_api_key=SecretStr('**********'))

In [ ]:
vectordb = FAISS.from_documents(documents, embedding=embedding)

In [27]:
retriever = vectordb.as_retriever()
retriever

NameError: name 'vectordb' is not defined

In [16]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever,"langsmith_search","search for information about langsmith. For any question about langsmith, you must use this tool")


In [17]:
retriever_tool.name

'langsmith_search'

## Creating another tool arxiv for research Paper 

In [28]:
## Arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)


In [29]:
arxiv_tool.name

'arxiv'

In [30]:
arxiv_tool.invoke("Tell me about the Machine Learning")

'Published: 2018-05-23\nTitle: Tell Me Something New: A New Framework for Asynchronous Parallel Learning\nAuthors: Julaiti Alafate, Yoav Freund\nSummary: We present a novel approach for parallel computati'

## Creating an DuckDuckGo Search tool for getting real time data or info 

In [31]:
from langchain_community.tools import DuckDuckGoSearchRun

web_tool = DuckDuckGoSearchRun()

web_tool.invoke("live weather update Makkah Madinah ?")

"Makkah Al Taif 20/12/2024 12:00 21:00 20/12/24 | 18:40 ... Lower Alert Update Alert Period View: 5 Yellow High winds Low horizontal visibility, Up to 40-49 km/h, Waves rising ... Get daily weather forecasts and severe early warnings by email for free. Update Alert Period View: 2 Yellow Light rain ... Makkah Makkah Al Mukarramah 22/12/2024 06:00 12:00 22/12/24 | 02:48 New Alert View: 8 Yellow Raised dust ... Get daily weather forecasts and severe early warnings by email for free. On Saturday, in Mecca, a cloudless sky and sunshiny weather are expected. The highest anticipated temperature is a hot 30°C (86°F), while the lowest temperature will be a refreshing 18°C (64.4°F). The predicted maximum temperature will match the common December peak of 32°C (89.6°F). Sunrise will be at 06:52 and sunset at 17:44; the daylight will last for 10h and 52min. مكة (Mecca) ☀ Weather forecast for 10 days, information from meteorological stations, webcams, sunrise and sunset, wind and precipitation maps

# Creating an agents using multiple tools

In [32]:
# getting all the three tools were make above
tools = [retriever_tool,arxiv_tool,wiki_tool,web_tool]

NameError: name 'retriever_tool' is not defined

In [42]:
tools

[Tool(name='langsmith_search', description='search for information about langsmith. For any question about langsmith, you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7722f212f4c0>, retriever=VectorStoreRetriever(tags=['FAISS', 'JinaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7722e4188320>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x7722f1e891c0>, retriever=VectorStoreRetriever(tags=['FAISS', 'JinaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7722e4188320>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), 

In [22]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0.6, 
    groq_api_key=os.getenv("GROQ_API_KEY"), 
    model_name="llama-3.3-70b-versatile"
    )


In [43]:
from langchain import hub
# Get the prompt to use - you can modify this!

# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages

# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

In [109]:
# # Custom prompt here
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# prompt1 = ChatPromptTemplate.from_messages(
#     [
#         ("system", "You are a helpful assistant"),
#         MessagesPlaceholder("chat_history", optional=True),
#         ("human", "{input}"),
#         MessagesPlaceholder("agent_scratchpad"),
#     ]
# )

In [54]:
# from langchain.chains import LLMChain

# llm_chain = LLMChain(llm=llm, prompt=prompt)

/tmp/ipykernel_5441/3642940915.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)


In [25]:
# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
#         ),
#         ("placeholder", "{chat_history}"),
#         ("human", "{input}"),
#         ("placeholder", "{agent_scratchpad}"),
#     ]
# )

In [110]:
# ### Agents

# from langchain.agents import create_openai_tools_agent, AgentExecutor
# # from langchain.agents import initialize_agent, AgentExecutor

# agent = create_openai_tools_agent(llm,tools,prompt)

In [26]:
# from langchain.agents import create_tool_calling_agent
# from langchain.agents import AgentExecutor


# agent = create_tool_calling_agent(llm, tools, prompt)

In [44]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [45]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [46]:
# agent_executor


In [30]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
To find information about LangChain, I should first consider what it is and where I can find relevant information. Given the tools available, I should start by searching for information about LangChain using the Wikipedia tool, as it is a general knowledge base that may have an entry on LangChain. If Wikipedia does not have sufficient information, I may need to consider other tools.

Action: wikipedia
Action Input: LangChainPage: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChainThought: The Wikipedia search provided a general overview of LangChain, but it might be beneficial to find more specific or detailed information, especially if it's related to its technical aspects, applications, or recent developments. Since LangChain is closely related to language models and their integration into various applic

{'input': 'what is LangChain?',
 'output': 'LangChain is a software framework that facilitates the integration of large language models (LLMs) into applications, and it is closely related to LangSmith, a platform for building production-grade LLM applications that allows for monitoring, evaluation, and deployment of LLMs. LangChain and LangSmith can be used together, with LangSmith providing tools for observability, evaluation, and prompt engineering, among other features.'}

In [31]:
agent_executor.invoke({"input":"what is langsmith? tell me more about langsmith"})



> Entering new AgentExecutor chain...
To answer the question about Langsmith, I first need to understand what Langsmith is and its purpose. 

Action: langsmith_search
Action Input: what is langsmithGet started with LangSmith | 🦜️🛠️ LangSmith






Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
With LangSmith you can:

LangSmith + LangChain OSSLangSmith integrates seamlessly with La

{'input': 'what is langsmith? tell me more about langsmith',
 'output': "Langsmith is a platform for building production-grade LLM applications, allowing users to monitor and evaluate their applications, and integrate with LangChain's open-source frameworks. It provides tools for creating datasets, defining interfaces, running evaluations, and analyzing results."}

In [32]:
try:
    response = agent_executor.invoke({"input": "Tell me about Machine Learning?"})
    print(response)
except Exception as e:
    print(f"Error: {e}")



> Entering new AgentExecutor chain...
To answer this question, I need to find a reliable source that provides a comprehensive overview of Machine Learning. 

Action: wikipedia
Action Input: Machine LearningPage: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genThought: The Wikipedia page provides a good introduction to Machine Learning, but I'd like to explore more about the current research and advancements in the field. 

Action: arxiv
Action Input: Machine LearningPublished: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton UniverThought: The Wikipedia page provides a good introduction to Machine Learning, and the arxiv search result provides some insight into current research and advancements in the field, 

In [47]:
try:
    response = agent_executor.invoke({"input": "live weather update Makkah Madinah"})
    print(response)
except Exception as e:
    print(f"Error: {e}")



> Entering new AgentExecutor chain...
To get the live weather update for Makkah and Madinah, I should look for current information, which is typically available through online news or search engines.

Action: duckduckgo_search
Action Input: live weather update Makkah MadinahError: https://links.duckduckgo.com/d.js?q=live+weather+update+Makkah+Madinah&kl=wt-wt&l=wt-wt&p=&s=0&df=y&vqd=4-61998003842299169156298642264826305796&bing_market=wt-WT&ex=-1 202 Ratelimit


In [33]:
# from langchain_core.messages import AIMessage, HumanMessage

# agent_executor.invoke(
#     {
#         "input": "Tell me about the Machine Learning",
#         "chat_history": [
#             HumanMessage(content="hi! my name is bob"),
#             AIMessage(content="Hello Bob! How can I assist you today?"),
#         ],
#     }
# )